In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = ( 
 SparkSession
 .builder
    .master("local[*]")
 .appName('spark_dataframe_api')
 .getOrCreate()
)

24/01/11 15:25:52 WARN Utils: Your hostname, sidnei-CBX resolves to a loopback address: 127.0.1.1; using 192.168.1.18 instead (on interface wlp1s0)
24/01/11 15:25:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/11 15:25:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/11 15:25:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./dados/precos-gasolina-etanol-11.csv')
)

df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [6]:
df_precos = (
    df
    .select('Estado - Sigla', 'Produto', 'Valor de Venda', 'Unidade de Medida')
    .withColumn(
        "Valor de Venda",
        F.regexp_replace(F.col("Valor de Venda"), ",", ".")
        .cast("float")
    )
)

In [8]:
df_precos.show(5)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AL|          GASOLINA|           6.3|       R$ / litro|
|            AL|GASOLINA ADITIVADA|           6.3|       R$ / litro|
|            AL|            ETANOL|          4.88|       R$ / litro|
|            AL|          GASOLINA|          6.19|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.29|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows



In [9]:
df_precos_analise = (
    df_precos
    .groupBy(
        F.col('Estado - Sigla'),
        F.col('Produto'),
        F.col('Unidade de Medida')
    )
    .agg(
        F.min(F.col("Valor de Venda")).alias('menor_valor'),
        F.max(F.col("Valor de Venda")).alias('maior_valor')
    )
    .withColumn(
        "diferenca",
        F.col('maior_valor') - F.col('menor_valor')
    )
    .orderBy('diferenca', ascending=False)
)

In [10]:
df_precos_analise.show(10)

+--------------+------------------+-----------------+-----------+-----------+---------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|diferenca|
+--------------+------------------+-----------------+-----------+-----------+---------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       4.69|       8.69|3.9999995|
|            SP|          GASOLINA|       R$ / litro|       4.49|       7.59|3.1000004|
|            PA|            ETANOL|       R$ / litro|       3.85|        6.6|     2.75|
|            SP|            ETANOL|       R$ / litro|       2.78|       5.34|2.5600002|
|            RS|            ETANOL|       R$ / litro|       3.88|       6.37|2.4899998|
|            PR|GASOLINA ADITIVADA|       R$ / litro|       5.19|       7.49|2.2999997|
|            MG|GASOLINA ADITIVADA|       R$ / litro|       5.09|       7.38|     2.29|
|            RS|GASOLINA ADITIVADA|       R$ / litro|       4.95|       7.13|2.1800003|
|            SC|            ETAN